<a href="https://colab.research.google.com/github/Ueeek/JP_EN_Transalation/blob/master/2_LSTM_jp_%3Een.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 内容

# 準備

## drive のマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import gc

In [0]:
base_dir = "drive/My Drive/ted_translation/".replace("/",os.sep)

In [0]:
corpus_file =base_dir+"data/jpn.txt"

In [0]:
emb_dir = base_dir+"trained_model/2lstm02.hdf5"

In [0]:
src_col="description_jp"
trg_col = "description_en"

 ## コーパスの読み込み

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import unicodedata
import string 
import re
import random

# 前処理

## word to index

In [0]:
SOS_token=1
EOS_token=0
UNK_token=2
max_features=5000

In [9]:
! pip3 install mosestokenizer 

In [0]:
import mosestokenizer as mos

In [0]:
class LangEn:
    def __init__(self,name):
        self.name= name
        self.word2index={"SOS":SOS_token,"EOS":EOS_token,"UNK":UNK_token}
        self.word2count={}
        self.index2word={SOS_token:"SOS",EOS_token:"EOS",UNK_token:"UNK"}
        self.n_words = 3
        self.tokenizer = mos.MosesTokenizer("en")
    def addSentence(self,sentence):
        for word in  self.tokenizer(sentence):
            self.addWord(word)
            
    def addWord(self,word):
        if word not in self.word2index  :
            if self.n_words >= max_features:
                return
            self.word2index[word] = self.n_words
            self.word2count[word]=1
            self.index2word[self.n_words]=word
            self.n_words+=1
        else:
            self.word2count[word]+=1
            
            
    def word2id(self,sentence,target=False):
        ret=[]
        if target:
            ret.append(self.word2index["SOS"])
        for word in self.tokenizer(sentence):
            try:
                word_id = self.word2index[word]
            except KeyError:
                word_id = self.word2index["UNK"]
            ret.append(word_id)
        ret.append(EOS_token)
        return ret
    
    
    def id2word(self,ids):
        ret=[]
        for id in ids:
            try:
                word = self.index2word[id]
            except KeyError:
                word = "UNK"
            ret.append(word)
        return ret
        

In [12]:
# colab　にmecabを入れる
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.4)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.7)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.2)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main

In [0]:
import MeCab
class LangJa:
    def __init__(self,name):
        self.name= name
        self.word2index={"SOS":SOS_token,"EOS":EOS_token,"UNK":UNK_token}
        self.word2count={}
        self.index2word={SOS_token:"SOS",EOS_token:"EOS",UNK_token:"UNK"}
        self.n_words = 3
        self.tagger = MeCab.Tagger("-Owakati")
        
    def addSentence(self,sentence):
        for word in self.tagger.parse(sentence).split():
            self.addWord(word)
            
    def addWord(self,word):
        if word not in self.word2index:
            if self.n_words >= max_features:
                return
            self.word2index[word] = self.n_words
            self.word2count[word]=1
            self.index2word[self.n_words]=word
            self.n_words+=1
        else:
            self.word2count[word]+=1
            
    def word2id(self,sentence,target=False):
        ret=[]
        if target:
            ret.append(self.word2index["SOS"])
        for word in self.tagger.parse(sentence).split():
            try:
                word_id=self.word2index[word]
            except KeyError:
                word_id = self.word2index["UNK"]
            ret.append(word_id)
        ret.append(EOS_token)
        return ret
        
        
    def id2word(self,ids):
        ret=[]
        for id in ids:
            try:
                word = self.index2word[id]
            except KeyError:
                word = "UNK"
            ret.append(word)
        return ret
                

## normalized

In [0]:
def unicodeToAscii(s):
    return " ".join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c)!="Mn")

In [0]:
def normalizeString(s):
    ret=[]
    for c in s.split():
        c = unicodeToAscii(c.lower())
        c = re.sub(r"([.!?])", r" \1", c)
        c = re.sub(r"[^a-zA-Z.!?]+", r" ", c)
        ret.append("".join(c.split()))
    return " ".join(ret)

In [0]:
def norm_jp(sentence):
    tagger = MeCab.Tagger("-Owakati")
    ret=[]
    for word in tagger.parse(sentence).split():
        word = re.sub(r"([、。])",r"\1",word)
        ret.append("".join(word.split()))
    return " ".join(ret)

In [0]:
def loadLangs():
    print("reading lines")
    data = pd.read_csv(corpus_file,sep="\t",names=[trg_col,src_col])
    data = data.sample(frac=1).reset_index(drop=True)
    data[src_col] = data[src_col].apply(lambda x:"".join(norm_jp(x)))
    data[trg_col] = data[trg_col].apply(lambda x:"".join(normalizeString(x)))
    return data[[src_col,trg_col]]

# Model

## keras

In [18]:
from keras.models import  Model
from keras.layers import Input,Dense,Add,Concatenate,Reshape,Dot,Softmax,Lambda
from keras.layers import LSTM,Embedding,Dropout,BatchNormalization
from keras.initializers import uniform
from keras import backend  as K
import numpy as np
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from math import ceil

Using TensorFlow backend.


### model config

In [0]:
class Seq2seq:
    def __init__(self,maxlen_e,maxlen_d,n_hidden,input_dim,emb_dim,output_dim):
        self.maxlen_e=maxlen_e
        self.maxlen_d=maxlen_d
        self.n_hidden=n_hidden
        self.input_dim=input_dim
        self.emb_dim=emb_dim
        self.output_dim=output_dim

    def create_model(self,use_enc_emb=False,use_dec_emb=False,encoder_emb_mat=None,decoder_emb_mat=None):
        print("#3 encoder")
        # encoder
        ## input
        encoder_input = Input(shape=(self.maxlen_e,),name="encoder_input")
        ## embedding
        if use_enc_emb:
            emb_input = Embedding(output_dim = self.emb_dim,input_dim = self.input_dim,weights=[encoder_emb_mat])(encoder_input)
            
        else:
            emb_input = Embedding(output_dim = self.emb_dim,input_dim = self.input_dim,embeddings_initializer=uniform(seed=SEED))(encoder_input)
        
        
        ## batch_norm
        emb_input = BatchNormalization(axis=-1)(emb_input)
        
        ## LSMT(forwarding)
        enc_fw1,state_h_fw1,state_c_fw1 = LSTM(self.n_hidden,
                                             name="encoder_LSTM_fw1",
                                             return_sequences=True,
                                             return_state = True)(emb_input)
        enc_LSTM_fw2 = LSTM(self.n_hidden,
                            name="encoder_LSTM_fw",
                           return_sequences=True,
                           return_state=True)
        enc_fw2,state_h_fw2,state_c_fw2 = enc_LSTM_fw2(enc_fw1)
        ## LSTM(backword)
        enc_bw1,state_h_bw1,state_c_bw1 = LSTM(self.n_hidden,
                                              name="encoder_LSTM_bw1",
                                              return_sequences=True,
                                              return_state=True)(emb_input)
        enc_bw2,state_h_bw2,state_c_bw2 = LSTM(self.n_hidden,
                                              name="encoder_LSTM_bw2",
                                              return_sequences=True,
                                              return_state=True)(enc_bw1)
        

         ## encoder_output
        encoder_outputs = Add()([enc_fw2,enc_bw2])
        state_h_1 = Add()([state_h_fw1,state_h_bw1])
        state_c_1 = Add()([state_c_fw1,state_c_bw1])
        state_h_2 = Add()([state_h_fw2,state_h_bw2])
        state_c_2 = Add()([state_c_fw2,state_c_bw2])
        
        encoder_states1 = [state_h_1,state_c_1]
        encoder_states2 = [state_h_2,state_c_2]
        ## encoder_output
       
        
       ## encoder_model
        encoder_model = Model(inputs=encoder_input, outputs=[encoder_outputs,state_h_1,state_c_1,state_h_2,state_c_2])
        
        print("#4 decoder")
        # decoder for train
        a_states1 = encoder_states1
        a_states2 = encoder_states2
        
        #define layers
        decode_LSTM1 = LSTM(self.n_hidden,
                             name="decode_LSTM1",
                             return_sequences=True,
                             return_state=True)
        decode_LSTM2 = LSTM(self.n_hidden,
                             name="decode_LSTM2",
                             return_sequences=True,
                             return_state=True)
        decoder_Dense=Dense(self.output_dim,
                           activation="softmax",
                           name="decoder_Dense")
        # deocoder
        decoder_inputs = Input(shape=(self.maxlen_d,),name="decoder_inputs")
        if use_dec_emb:
            dec_emb =Embedding(output_dim = self.emb_dim,
                              input_dim = self.output_dim,
                              weights=[decoder_emb_mat] )(decoder_inputs)
        else:
            dec_emb =Embedding(output_dim = self.emb_dim,
                              input_dim = self.output_dim,
                              embeddings_initializer=uniform(seed=SEED))(decoder_inputs)
            
        decoder_input = BatchNormalization(axis=-1)(dec_emb)
        dec_input=decoder_input
        decoder_lstm1,state_s_1,state_c_1 = decode_LSTM1(dec_input,initial_state=encoder_states1)
        decoder_lstm2,state_s_2,state_c_2 = decode_LSTM2(decoder_lstm1,initial_state=encoder_states2)
        
    
      
        decoder_outputs = decoder_Dense(decoder_lstm2)
        print("#5")
        
        model = Model(inputs=[encoder_input,decoder_inputs],outputs=decoder_outputs)
        model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["categorical_accuracy"])
        
        print("#6")
        #decoder for generate translation
        decoder_state_input_h_1 = Input(shape=(self.n_hidden,),name='input_h_1')
        decoder_state_input_c_1 = Input(shape=(self.n_hidden,),name='input_c_1')
        decoder_state_input_h_2 = Input(shape=(self.n_hidden,),name='input_h_2')
        decoder_state_input_c_2 = Input(shape=(self.n_hidden,),name='input_c_2')
      
      

      ##　上のやつをまとめる
        decoder_states_inputs_1 = [decoder_state_input_h_1, decoder_state_input_c_1]
        decoder_states_inputs_2 = [decoder_state_input_h_2, decoder_state_input_c_2] 
        decoder_states_inputs=[decoder_state_input_h_1, decoder_state_input_c_1,
                               decoder_state_input_h_2, decoder_state_input_c_2]
        
        
        ##LSTM
        decoder_lstm_1,state_h_1,state_c_1 = decode_LSTM1(dec_input,initial_state=decoder_states_inputs_1)
        decoder_lstm_2,state_h_2,state_c_2 = decode_LSTM2(decoder_lstm_1,initial_state=decoder_states_inputs_2)
        
        decoder_states = [state_h_1,state_c_1,state_h_2,state_c_2]
        
        
        
        
        print("#7")
        #output
        decoder_outputs=decoder_Dense(decoder_lstm_2)
        
        #decoder model
        decoder_model=Model(
            [decoder_inputs]+decoder_states_inputs,
            [decoder_outputs]+decoder_states)
        
        return model,encoder_model ,decoder_model
    
    
    ## train
    def train(self,e_input,d_input,target,emb_dir,batch_size=1,epochs=1,validation_split=0):
        print("#1 train procedure start")
        model,_,_ = self.create_model(use_enc_emb=True,use_dec_emb=True,encoder_emb_mat=en_embedding_matrix,decoder_emb_mat=jp_embedding_matrix)
        model.summary()
        
        if os.path.isfile(emb_dir): #モデルの学習済みパラメータ
            print("2-1? load param")
            model.load_weights(emb_dir)
        else:
            print("no_emb")
        print("#6 start training")
        
        target_categorical = np_utils.to_categorical(output_target_padded,self.output_dim)
        
       
        model.fit([e_input,d_input],target_categorical,epochs=epochs,batch_size=batch_size,validation_split=validation_split)
            
        print("#9 save_param")
        model.save_weights(emb_dir)
        return model    
        
       
          

    ## 翻訳文生成
    def translate(self,e_input,length,encoder_model,decoder_model):
        #encode input to vec
        encoder_outputs,state_h_1,state_c_1,state_h_2,state_c_2 = encoder_model.predict(e_input)
        states_values=[state_h_1,state_c_1,state_h_2,state_c_2]
        
        
        #first token
        target_seq=np.zeros((1,1))
        target_seq[0,0] = SOS_token #id of SOS (start of sentence)
        
        decoded_sentence=[]
        for i in range(0,length):
            output_tokens,h1,c1,h2,c2 = decoder_model.predict([target_seq]+states_values)
            
            sampled_token_index=np.argmax(output_tokens[0,0,:])
            #print("token->",sampled_token_index)
            if sampled_token_index==EOS_token: #EOS
                decoded_sentence.append(EOS_token)
                break
            else:
                target_seq[0,0] = sampled_token_index
                states_values =[h1,c1,h2,c2]
                decoded_sentence.append(sampled_token_index)
        return decoded_sentence                                    
    
    
    
    def translate_demo(self,src_data_id_seq,param_dir):
        #model,encoder_model,decoder_model = self.create_model(use_enc_emb=True,use_dec_emb=True,encoder_emb_mat=en_embedding_matrix,decoder_emb_mat=jp_embedding_matrix)
        model,encoder_model,decoder_model = self.create_model()
        model.load_weights(param_dir)
       
        ret=[]
        for src in src_data_id_seq:
            id_seq_mat = np.array([src])
            pred_id_padded = sequence.pad_sequences(id_seq_mat,maxlen=MAX_LENGTH,padding="post",truncating="post")
            pred=Generate_model.translate(pred_id_padded,20,encoder_model,decoder_model)
            ret.append(pred)
        return ret
        
        

### model visualizer

In [0]:
#from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot

In [0]:
#SVG(model_to_dot(training_model).create(prog='dot', format='svg'))

In [0]:
#SVG(model_to_dot(encoder_model).create(prog='dot', format='svg'))

In [0]:
#SVG(model_to_dot(decoder_model).create(prog='dot', format='svg'))

# train

In [24]:
data=loadLangs()

reading lines


In [25]:
data[src_col].apply(lambda x:len(x.split())).describe()

count    43953.000000
mean         9.875867
std          4.008849
min          1.000000
25%          7.000000
50%          9.000000
75%         12.000000
max         58.000000
Name: description_jp, dtype: float64

In [0]:
MAX_LENGTH= 20
def filterPair(data):
    return data[data.apply(lambda x:len(x[trg_col].split(" ")),axis=1)<MAX_LENGTH]

In [0]:
data = filterPair(data)
val_data = data[10000:10100]
data = data[:15000]

In [0]:
JP_lang = LangEn(src_col)
EN_lang = LangJa(trg_col)

In [0]:
for s in data[trg_col]:
    EN_lang.addSentence(s)

In [0]:
for s in data[src_col]:
    JP_lang.addSentence(s)

## create emb mat

### load

In [0]:
en_W2V_FILE = "./drive/My Drive/util_data/GoogleNews-vectors-negative300.bin.gz"
jp_W2V_FILE="./drive/My Drive/util_data/ja_data/ja.bin"

In [0]:
emb_dim=300

In [0]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

In [0]:
en_word2vec= KeyedVectors.load_word2vec_format(en_W2V_FILE,binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
jp_word2vec= model = Word2Vec.load(jp_W2V_FILE)

### create

In [0]:
en_EMBEDDING_DIM=emb_dim
vocabulary_size=min(EN_lang.n_words,max_features)
en_embedding_matrix = np.zeros((vocabulary_size, en_EMBEDDING_DIM))
print("voc->",vocabulary_size)
cnt=0
for word, i in EN_lang.word2index.items():
    if   i==0 or i==1 or i ==2:
        continue
    try:
        en_embedding_vector = en_word2vec[word]
        en_embedding_matrix[i] = en_embedding_vector
    except KeyError:
        cnt+=1
        en_embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25),en_EMBEDDING_DIM)
print("UNK_rate",cnt/i)

In [0]:
jp_EMBEDDING_DIM=emb_dim
vocabulary_size=min(JP_lang.n_words,max_features)
jp_embedding_matrix = np.zeros((vocabulary_size, jp_EMBEDDING_DIM))
print("voc->",vocabulary_size)
cnt=0
for word, i in JP_lang.word2index.items():
    if   i==0 or i==1 or i ==2:
        continue
    try:
        jp_embedding_vector = jp_word2vec[word]
        jp_embedding_matrix[i] = jp_embedding_vector
    except KeyError:
        cnt+=1
        jp_embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25),jp_EMBEDDING_DIM)
print("UNK/rate->",cnt/i)

In [0]:
#RAMの節約
del jp_word2vec
del en_word2vec
gc.collect()

## input の加工

In [0]:
input_source_lang=data[src_col].apply(lambda x:JP_lang.word2id(x))
input_target_lang=data[trg_col].apply(lambda x:EN_lang.word2id(x,target=True))
output_target_lang=data[trg_col].apply(lambda x:EN_lang.word2id(x))

In [0]:
data[src_col][:10]

Paddning


In [0]:
from keras.preprocessing import sequence

In [0]:
input_source_padded=sequence.pad_sequences(input_source_lang,maxlen=MAX_LENGTH,padding="post",truncating="post")
input_target_padded=sequence.pad_sequences(input_target_lang,maxlen=MAX_LENGTH,padding="post",truncating="post")
output_target_padded=sequence.pad_sequences(output_target_lang,maxlen=MAX_LENGTH,padding="post",truncating="post")

In [0]:
batch_size = 128
epochs=5
input_dim = EN_lang.n_words
output_dim = JP_lang.n_words
n_hidden=int(emb_dim+100)
SEED=200000

In [0]:
training_model= Seq2seq(maxlen_e=MAX_LENGTH,
                              maxlen_d=MAX_LENGTH,
                              n_hidden=n_hidden,
                              input_dim=input_dim,
                              emb_dim = emb_dim,
                              output_dim=output_dim)

In [0]:
 training_model.train(e_input=input_source_padded,
                      d_input=input_target_padded,
                      target = output_target_padded,
                      emb_dir=emb_dir,
                      epochs=epochs,
                      batch_size=batch_size,
                      validation_split=0.1)

In [0]:
del training_model
gc.collect()

# test

In [0]:
val_data.head(10)

In [0]:
val_data_id = val_data[src_col].apply(lambda x:JP_lang.word2id(x))

In [0]:
val_data_id[:10]

In [0]:
Generate_model= Seq2seq(maxlen_e=MAX_LENGTH,
                              maxlen_d=1,
                              n_hidden=n_hidden,
                              input_dim=input_dim,
                             emb_dim = emb_dim,
                              output_dim=output_dim)

In [0]:
ret=Generate_model.translate_demo(val_data_id,param_dir=emb_dir)

In [0]:
for src,pred,target in zip(val_data[src_col],ret,val_data[trg_col]):
    print("src->",src)
    print("pred->",EN_lang.id2word(pred))
    print("ans->",target)
    print("------------------")